# **<h1><center><span style="color:#3775a8"><u>Projet Analyse des données massives </u></span></center></h1>**

 #### AYENA Mahougnon Mariel Ulrich

In [13]:
#!pip install pyspark

In [14]:
#!pip install nltk

In [2]:
#import nltk
#nltk.download('vader_lexicon')

In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover,CountVectorizer

In [27]:


# Créer une session Spark
spark = SparkSession.builder.appName("Lecture JSON").getOrCreate()

# Charger un fichier JSON dans un DataFrame
train = spark.read.json("./train.json")
test = spark.read.json("./test.json")

In [28]:
train.show(5)

+--------------------+--------+
|             message|polarity|
+--------------------+--------+
|! Comment était l...|       0|
|! d'accord! Va-t-...|       0|
|!!! Taihen desu n...|       0|
|!!!! Auto-dj .. c...|       0|
|!!!! Ce n'est que...|       0|
+--------------------+--------+
only showing top 5 rows



In [29]:
test.show(5)

+--------------------+--------+
|             message|polarity|
+--------------------+--------+
|! Et si affamé ma...|       0|
|! Identica présen...|       0|
|! Je vais enfin m...|       0|
|!?!? C'est un jou...|       0|
|"Easy" qu Le plan...|       0|
+--------------------+--------+
only showing top 5 rows



In [30]:
train_rows=train.count()
test_rows=test.count()
print("Total train :",train_rows)
print("Total test :", test_rows)


Total train : 128401
Total test : 76759


In [31]:
train.select("polarity").distinct().show()

+--------+
|polarity|
+--------+
|       0|
|       4|
+--------+



In [32]:
train.groupBy("polarity").count().show()

+--------+-----+
|polarity|count|
+--------+-----+
|       0|61475|
|       4|66926|
+--------+-----+



In [33]:
#test.select("polarity").distinct().show()

In [34]:
test.groupBy("polarity").count().show()

+--------+-----+
|polarity|count|
+--------+-----+
|       0|36890|
|       4|39869|
+--------+-----+



# 2- Prétraitement des données

### 2.1- 1ére méthode d'extraction de caractéristiques :  HashingTF

In [35]:
# Créez l'objet RegexTokenizer pour la tokenization
tokenizer = Tokenizer(inputCol="message", outputCol="SentimentWords")
# Créez l'objet StopWordsRemover pour supprimer les mots vides
stopwords_remover= StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="MeaningfulWords")
# Créez l'objet HashingTF pour convertir les mots en vecteurs de compte
hashTF = HashingTF(inputCol=stopwords_remover.getOutputCol(), outputCol="features")


In [36]:
##Création d'un pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[tokenizer, stopwords_remover ,hashTF])

# application du  pipeline aux documents des données d'entrainement 
pipelineFit = pipeline.fit(train)
train_n = pipelineFit.transform(train)
# Convertir "polarity" en valeurs numériques
train_n= train_n.withColumn("polarity",train_n["polarity"].cast("int"))
train_n.show(5)

+--------------------+--------+--------------------+--------------------+--------------------+
|             message|polarity|      SentimentWords|     MeaningfulWords|            features|
+--------------------+--------+--------------------+--------------------+--------------------+
|! Comment était l...|       0|[!, comment, étai...|[!, comment, étai...|(262144,[18632,29...|
|! d'accord! Va-t-...|       0|[!, d'accord!, va...|[!, d'accord!, va...|(262144,[10096,22...|
|!!! Taihen desu n...|       0|[!!!, taihen, des...|[!!!, taihen, des...|(262144,[20025,38...|
|!!!! Auto-dj .. c...|       0|[!!!!, auto-dj, ....|[!!!!, auto-dj, ....|(262144,[4231,418...|
|!!!! Ce n'est que...|       0|[!!!!, ce, n'est,...|[!!!!, ce, n'est,...|(262144,[18868,29...|
+--------------------+--------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [37]:
# application du  pipeline aux documents des données de test.
test_n = pipelineFit.transform(test)
test_n= test_n.withColumn("polarity",test_n["polarity"].cast("int"))

# 3- Modélisation

### 3.1- Régression Logistique

In [38]:
#Entrainement du modèle
lr = LogisticRegression(labelCol="polarity", featuresCol="features", maxIter=10, regParam=0.01)
model_lr = lr.fit(train_n)

In [39]:
#Prediction sur les données test
predictions = model_lr.transform(test_n)
predictions.printSchema()

root
 |-- message: string (nullable = true)
 |-- polarity: integer (nullable = true)
 |-- SentimentWords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- MeaningfulWords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [40]:
predictions.select("MeaningfulWords", "prediction", "polarity").show(5)

+--------------------+----------+--------+
|     MeaningfulWords|prediction|polarity|
+--------------------+----------+--------+
|[!, et, si, affam...|       0.0|       0|
|[!, identica, pré...|       4.0|       0|
|[!, je, vais, enf...|       0.0|       0|
|[!?!?, c'est, un,...|       0.0|       0|
|["easy", qu, le, ...|       4.0|       0|
+--------------------+----------+--------+
only showing top 5 rows



In [41]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [42]:
##Score de la prédiction
evaluator = MulticlassClassificationEvaluator(labelCol="polarity", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy: ", accuracy)

Accuracy:  0.7346500084680624


### 3.2- Random Forest

In [142]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="polarity",featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32)
model_rf = rf.fit(train_n)
predictions_n = model_rf.transform(test_n)

In [143]:
evaluator = MulticlassClassificationEvaluator(labelCol="polarity", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions_n)
print("Accuracy: ", accuracy)

Accuracy:  0.5195351685144413


### 3.3- Naive Bayes

In [139]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(labelCol="polarity", featuresCol="features",smoothing=1)
model_nb = nb.fit(train_n)
predictions_nn = model_nb.transform(test_n)

In [140]:
evaluator = MulticlassClassificationEvaluator(labelCol="polarity", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions_nn)
print("Accuracy: ", accuracy)

Accuracy:  0.378756888443049


**Commentaire**:

Lors de l'application des trois modèles, nous avons obtenu les résultats suivants : le modèle de régression logistique a une précision de 0.7346 pour prédire les polarités sur les données de test, le Random Forest a une précision de 0.5195, tandis que le modèle Naive Bayes a une précision de 0.3787. Nous avons constaté que parmi ces trois modèles, c'est le modèle de régression logistique qui a obtenu le meilleur score. Par conséquent, nous allons effectuer une validation croisée sur ce modèle en utilisant plusieurs paramètres afin de sélectionner les meilleurs.

### 3.4- Cross-validation avec la régression Logistique

In [19]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import  MulticlassClassificationEvaluator

lr = LogisticRegression(labelCol="polarity", featuresCol="features")

# Création de ParamGrid pour la  Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.1,0.3])  # régularisation  des paramètres
             .addGrid(lr.elasticNetParam, [0.0,0.1])  #  paramètres de Elastic Net  (Ridge = 0)
             .addGrid(lr.maxIter, [10,20])  #  Nombre d'itération
             .build())

evaluator = MulticlassClassificationEvaluator(labelCol="polarity", predictionCol="prediction", metricName="accuracy")

# Création de 5-fold CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

cvModel = cv.fit(train_n)
predictions = cvModel.transform(test_n)

# Evaluatin du meilleur modèle
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)


Accuracy: 0.75473885798408


**Commentaire**

Amélioration du score après cross validation sur le modèle de régression

# 4- Autres méthodes d'extraction 

### 4.1-    2éme méthode d'extraction de caractéristiques :  Term Frequency - Inverse Document Frequency (TF-IDF)

In [43]:
from pyspark.ml.feature import HashingTF, Tokenizer,RegexTokenizer, StopWordsRemover,IDF,StringIndexer

In [44]:
tokenizer = Tokenizer(inputCol="message", outputCol="SentimentWords")
stopwords_remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="MeaningfulWords")
hashing_tf = HashingTF(inputCol=stopwords_remover.getOutputCol(), outputCol="features")
idf = IDF(inputCol=hashing_tf.getOutputCol(), outputCol="idf_features")
#label_string_idx = StringIndexer(inputCol="polarity", outputCol="polarity_index")


In [45]:

pipeline = Pipeline(stages=[tokenizer, stopwords_remover, hashing_tf, idf])

In [46]:
pipelineFit2 = pipeline.fit(train)
train_n2 = pipelineFit.transform(train)
train_n2= train_n2.withColumn("polarity",train_n2["polarity"].cast("int"))
train_n2.show(5)

+--------------------+--------+--------------------+--------------------+--------------------+
|             message|polarity|      SentimentWords|     MeaningfulWords|            features|
+--------------------+--------+--------------------+--------------------+--------------------+
|! Comment était l...|       0|[!, comment, étai...|[!, comment, étai...|(262144,[18632,29...|
|! d'accord! Va-t-...|       0|[!, d'accord!, va...|[!, d'accord!, va...|(262144,[10096,22...|
|!!! Taihen desu n...|       0|[!!!, taihen, des...|[!!!, taihen, des...|(262144,[20025,38...|
|!!!! Auto-dj .. c...|       0|[!!!!, auto-dj, ....|[!!!!, auto-dj, ....|(262144,[4231,418...|
|!!!! Ce n'est que...|       0|[!!!!, ce, n'est,...|[!!!!, ce, n'est,...|(262144,[18868,29...|
+--------------------+--------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [47]:
test_n2 = pipelineFit2.transform(test)
test_n2= test_n2.withColumn("polarity",test_n2["polarity"].cast("int"))

### 4.2- Régression Logistique

In [48]:
# Model d'entraînement
lr = LogisticRegression(labelCol="polarity", featuresCol="features", maxIter=20, regParam=0.3)
model_lr = lr.fit(train_n2)

In [49]:
#Prediction sur les données test
predictions = model_lr.transform(test_n2)

In [50]:
evaluator = MulticlassClassificationEvaluator(labelCol="polarity", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy: ", accuracy)

Accuracy:  0.7546997746192629


### 4.3-   3ème méthode d'extraction de caractéristiques :  CountVectorizer

In [51]:
# Créez l'objet RegexTokenizer pour la tokenization
#tokenizer =  RegexTokenizer(inputCol="message", outputCol="SentimentWords", pattern="\\W")
tokenizer =  Tokenizer(inputCol="message", outputCol="SentimentWords")
# Créez l'objet StopWordsRemover pour supprimer les mots vides
stopwords_remover= StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="MeaningfulWords")
# Créez l'objet CountVectorizer pour extraire les vecteurs de compte
count_vectorizer = CountVectorizer(inputCol=stopwords_remover.getOutputCol(), outputCol="features")
#label_string_idx = StringIndexer(inputCol="polarity", outputCol="polarity_index")


In [52]:
pipeline = Pipeline(stages=[tokenizer, stopwords_remover, count_vectorizer])

In [53]:
pipelineFit3 = pipeline.fit(train)
train_n3 = pipelineFit3.transform(train)
train_n3= train_n3.withColumn("polarity",train_n3["polarity"].cast("int"))
#train_n3.printSchema()
train_n3.show(5)

+--------------------+--------+--------------------+--------------------+--------------------+
|             message|polarity|      SentimentWords|     MeaningfulWords|            features|
+--------------------+--------+--------------------+--------------------+--------------------+
|! Comment était l...|       0|[!, comment, étai...|[!, comment, étai...|(101862,[3,21,87,...|
|! d'accord! Va-t-...|       0|[!, d'accord!, va...|[!, d'accord!, va...|(101862,[4,6,7,8,...|
|!!! Taihen desu n...|       0|[!!!, taihen, des...|[!!!, taihen, des...|(101862,[0,17,43,...|
|!!!! Auto-dj .. c...|       0|[!!!!, auto-dj, ....|[!!!!, auto-dj, ....|(101862,[5,8,9,20...|
|!!!! Ce n'est que...|       0|[!!!!, ce, n'est,...|[!!!!, ce, n'est,...|(101862,[4,5,6,10...|
+--------------------+--------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [54]:
#pipelineFit3 = pipeline.fit(test)
test_n3 = pipelineFit3.transform(test)
test_n3= test_n3.withColumn("polarity",test_n3["polarity"].cast("int"))
test_n3.show(5)

+--------------------+--------+--------------------+--------------------+--------------------+
|             message|polarity|      SentimentWords|     MeaningfulWords|            features|
+--------------------+--------+--------------------+--------------------+--------------------+
|! Et si affamé ma...|       0|[!, et, si, affam...|[!, et, si, affam...|(101862,[0,3,6,7,...|
|! Identica présen...|       0|[!, identica, pré...|[!, identica, pré...|(101862,[3,7,21,2...|
|! Je vais enfin m...|       0|[!, je, vais, enf...|[!, je, vais, enf...|(101862,[0,1,12,1...|
|!?!? C'est un jou...|       0|[!?!?, c'est, un,...|[!?!?, c'est, un,...|(101862,[0,1,2,4,...|
|"Easy" qu Le plan...|       0|["easy", qu, le, ...|["easy", qu, le, ...|(101862,[2,4,14,1...|
+--------------------+--------+--------------------+--------------------+--------------------+
only showing top 5 rows



### 4.4-   Régression Logistique

In [55]:
#Training Model
lr = LogisticRegression(labelCol="polarity", featuresCol="features", maxIter=20, regParam=0.3)
model_lr = lr.fit(train_n3)

In [56]:
#Prediction
predictions = model_lr.transform(test_n3)
#predictions.printSchema()

In [57]:
prediction_final = predictions.select("SentimentWords", "prediction", "polarity")
prediction_final.show(5)

+--------------------+----------+--------+
|      SentimentWords|prediction|polarity|
+--------------------+----------+--------+
|[!, et, si, affam...|       0.0|       0|
|[!, identica, pré...|       4.0|       0|
|[!, je, vais, enf...|       0.0|       0|
|[!?!?, c'est, un,...|       0.0|       0|
|["easy", qu, le, ...|       0.0|       0|
+--------------------+----------+--------+
only showing top 5 rows



In [58]:
evaluator = MulticlassClassificationEvaluator(labelCol="polarity", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy: ", accuracy)

Accuracy:  0.7611615576023658


**Commentaire**

En utilisant la méthode d'extraction de caractéristiques **CountVectorizer** et la régression logistique avec les meilleurs paramètres, nous avons un score de 76,11% . Ce qui est suppérieur à tous les autres scores trouvés.Donc nous allons garder cette technique d'extration pour faire les prédiction sur le fichier **noclass**.


### 4-5  Prédiction sur le fichier noclass.json

Pour cette partie nous allons faire les prédictions avec le meilleur modèle. C'est à dire le modèle le plus performant en terme de score. Dans notre c'est le modèle de régression logistique avec les bons paramètres.

In [67]:
##Lecture du fichier
noclass = spark.read.json("./noclass.json")

In [68]:
noclass.show(5)

+--------------------+
|             message|
+--------------------+
|"Dans ganga" Ne v...|
|"Dieu elton" vous...|
|"I was up up the ...|
|"Quasi" toute la ...|
|# $ & Amp; * # vi...|
+--------------------+
only showing top 5 rows



In [69]:
noclass_n = pipelineFit3.transform(noclass)
noclass_n.show(5)

+--------------------+--------------------+--------------------+--------------------+
|             message|      SentimentWords|     MeaningfulWords|            features|
+--------------------+--------------------+--------------------+--------------------+
|"Dans ganga" Ne v...|["dans, ganga", n...|["dans, ganga", n...|(101862,[1,3,4,6,...|
|"Dieu elton" vous...|["dieu, elton", v...|["dieu, elton", v...|(101862,[0,1,5,6,...|
|"I was up up the ...|["i, was, up, up,...|["i, half, night,...|(101862,[1,5,6,17...|
|"Quasi" toute la ...|["quasi", toute, ...|["quasi", toute, ...|(101862,[1,3,7,14...|
|# $ & Amp; * # vi...|[#, $, &, amp;, *...|[#, $, &, amp;, *...|(101862,[0,6,7,12...|
+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



#####  Apllication de la régression Logistique au données du fichier noclass

In [70]:
#Prediction
predictions_noclass = model_lr.transform(noclass_n)
predictions_noclass.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|             message|      SentimentWords|     MeaningfulWords|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|"Dans ganga" Ne v...|["dans, ganga", n...|["dans, ganga", n...|(101862,[1,3,4,6,...|[6.65713799494093...|[0.50338760506601...|       0.0|
|"Dieu elton" vous...|["dieu, elton", v...|["dieu, elton", v...|(101862,[0,1,5,6,...|[7.78662181371817...|[0.90656960881930...|       0.0|
|"I was up up the ...|["i, was, up, up,...|["i, half, night,...|(101862,[1,5,6,17...|[6.97102532189767...|[0.65505020154945...|       0.0|
|"Quasi" toute la ...|["quasi", toute, ...|["quasi", toute, ...|(101862,[1,3,7,14...|[6.84104638127328...|[0.59419197081360...|       0.0|
|# $ & Amp; * # vi...|[#, $

In [74]:
prediction_final_noclass = predictions_noclass.select("message", "prediction")
prediction_final_noclass.show(5)

+--------------------+----------+
|             message|prediction|
+--------------------+----------+
|"Dans ganga" Ne v...|       0.0|
|"Dieu elton" vous...|       0.0|
|"I was up up the ...|       0.0|
|"Quasi" toute la ...|       0.0|
|# $ & Amp; * # vi...|       0.0|
+--------------------+----------+
only showing top 5 rows



### 4-6 Exportation du fichier Noclass avec les prédiction

In [ ]:
# Fichier des prédictions
predictions = prediction_final_noclass.withColumn("prediction", col("prediction").cast("integer"))
predictions_json = predictions.select('message','prediction').coalesce(1)
predictions_json.write.json("./predictions.json")
